In [30]:
import requests
import uuid
from urllib.parse import urljoin, urlencode, urlparse, parse_qs, urlunparse
import time
from itertools import islice, tee

In [3]:
BASE_URL = "https://www.taptap.cn/webapiv2/"
USER_AGENT = (
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
    "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36"
)

PLATFORM = {
    "ios": "iOS",
    "android": "Android"
}

In [4]:
class TapTapClient:
    def __init__(self):
        self.session = requests.Session()
        self.session.headers.update({
            "user-agent": USER_AGENT
        })

    def _build_url(self, path: str, params: dict) -> str:
        full_url = urljoin(BASE_URL, path)
        parsed = urlparse(full_url)
        query = parse_qs(parsed.query)

        # Convert to snake_case if needed here, skipping for now
        for k, v in params.items():
            query[k] = v

        # Add X-UA header in query params
        platform = params.get("platform", "android").lower()
        device_platform = PLATFORM.get(platform, "Android")
        query["X-UA"] = (
            f"V=1&PN=WebApp&LANG=zh_CN&VN_CODE=102&LOC=CN&PLT=PC&"
            f"DS={device_platform}&UID={uuid.uuid4()}&OS=Mac+OS&OSV=10.15.7&DT=PC"
        )

        encoded_query = urlencode(query, doseq=True)
        final_url = urlunparse(parsed._replace(query=encoded_query))
        return final_url

    def get(self, path: str, params: dict = None):
        if params is None:
            params = {}

        url = self._build_url(path, params)
        response = self.session.get(url)
        response.raise_for_status()

        data = response.json()
        if not data.get("success", False):
            raise Exception("Request failed")

        return data.get("data", {})

    def list(self, path: str, params: dict = None):
        if params is None:
            params = {}
        params.setdefault("from", 0)
        params.setdefault("limit", 10)

        while True:
            data = self.get(path, params)
            total = data.get("total", 0)
            items = data.get("list", [])
            next_page = data.get("next_page")

            for item in items:
                yield item

            if not next_page or params["from"] + len(items) >= total:
                break

            params["from"] += len(items)
            time.sleep(1)

    def get_app(self, app_id, **params):
        return self.get("app/v4/detail", {"id": app_id, **params})

    def list_apps(self, type_name="reserve", **params):
        params.setdefault("type_name", type_name)
        for row in self.list("app-top/v2/hits", params):
            if not row.get("is_add") and row.get("type") == "app":
                yield row["app"]

    def list_reviews(self, app_id, sort="new", **params):
        params.update({
            "app_id": app_id,
            "sort": sort
        })
        for row in self.list("review/v2/list-by-app", params):
            if row.get("type") == "moment":
                yield row["moment"]

# Execute

In [5]:
client = TapTapClient()

In [6]:
# Get app details
app_data = client.get_app(209601)
print(app_data["title"])

长安幻想


In [7]:
app_data

{'id': 209601,
 'identifier': 'com.cahx.sy',
 'itunes_id': '1555236168',
 'title': '长安幻想',
 'title_labels': [],
 'icon': {'url': 'https://img.tapimg.com/market/images/39928f495aa9d307fb07c934f10f46b4.png/appicon?t=1',
  'medium_url': 'https://img.tapimg.com/market/images/39928f495aa9d307fb07c934f10f46b4.png/appicon_m?t=1',
  'small_url': 'https://img.tapimg.com/market/images/39928f495aa9d307fb07c934f10f46b4.png/appicon_s?t=1',
  'original_url': 'https://img.tapimg.com/market/images/39928f495aa9d307fb07c934f10f46b4.png',
  'original_format': 'png',
  'width': 270,
  'height': 270,
  'color': '0x748799',
  'original_size': 467360},
 'style': 0,
 'update_time': 1743157406,
 'hidden_button': False,
 'has_moment_rec': False,
 'is_deny_minors': False,
 'is_exclusive': False,
 'app_videos': [{'type': 'app_detail',
   'id': 4639098,
   'thumbnail': {'url': 'https://img.tapimg.com/market/images/d4f2b8e9bdda1b10492f78397db062d0.jpg?imageView2/0/w/1080/h/608/format/jpg/interlace/1/ignore-error/1&

In [9]:
# List top apps
top_apps = client.list_apps()

In [14]:
for app in client.list_apps():
    print(app["title"])

异人之下
三千幻世
代号：界
掌门下山
黑色信标
胜利女神：新的希望
杖剑传说
银与绯
洛克王国：世界
梦的第七章
英勇之地
盗墓笔记：启程
暴吵萌厨
龙魂旅人
Project Rene-模拟人生
七日世界
渔帆暗涌
望月
怪物之家2：勋章
口袋吉伊卡哇 (Chiikawa Pocket)
赛尔号巅峰之战
荒野起源
我独自升级 Arise
凝渊
异环
骗子酒馆
唱舞星计划
华夏千秋
二重螺旋
代号：诡秘
鹅鸭杀
星痕共鸣
魔法少女まどか☆マギカ Magia Exedra
蜡笔小新之小帮手大作战
镭明闪击
百相世界
落日山丘
明日方舟：终末地
无限大
此间山海（TapTap测试版）
代号：芙娅之魂
群星纪元
宝可梦 冠军
哀鸿：城破十日记
逆战：未来
火山的女儿
可口的咖啡
腐蚀(Rust)手游
代号：撤离（TapTap测试版）
Trainee Death Simulator
幻想少女公会
从军
未来之役
斗罗大陆：猎魂世界
波斯王子：失落的王冠
伍六七：暗影交锋
远光84
洛伊的移动要塞
崩溃大陆2
潜水员戴夫
苍蓝避风港
对决！剑之川
白日梦想屋
怪物乐土
Notanote（TapTap测试版）
踏风行
蓝色星原：旅谣
地狱之吻
重生之最强输出
方舟：生存进化
代号：JUMP
纸上谈亲
卡拉彼丘
杀青
米姆米姆哈
奥特曼：超时空英雄
NBA 2K25梦幻球队
代号:速降(Descenders)
Mixlody
不良英雄谭
菜鸡梦想家
桃源记2
山海仙路
山海进化录
辉烬Embers
饥困荒野
我在末日囤物资
帝国游戏
花花与幕间剧
斗罗大陆：诛邪传说
幻兽帕鲁手游
烣境
火影忍者：木叶高手
数码宝贝：源码
边狱巴士
群星低语-Whispers from the Star
Arcaea
你来嘛英雄
星际战甲Warframe手游
百面千相
境·界 刀鸣
筑城与探险
PuffPals: Island Skies
Astropulse
荒原曙光
失落城堡2（TapTap测试版）
萤火夜话
冰冬冬小镇
NOeSIS_诉说谎言的记忆之物语
天空岛传说
古镇闲居
咒印链接
蛙蛙豹豹的树屋
火影忍者：究极忍者风暴
虚拟化学实验室
武娘
Break My Case
彩虹六号
星之破晓
现代战舰
源序空间
使命召唤®：战争地带™手游
魔法：精灵世界
TIRBE NINE：战极死游
心

In [33]:
# List reviews for a game
reviews = client.list_reviews(209601)

In [21]:
type(reviews)

generator

In [27]:
preview = list(islice(reviews, 5))  # First 5 items

In [28]:
preview

[{'id_str': '656978632270416405',
  'created_time': 1743688712,
  'edited_time': 1743688712,
  'publish_time': 1743688712,
  'commented_time': 1743688712,
  'author': {'user': {'id': 23562097,
    'name': '音羽',
    'avatar': 'https://img3.tapimg.com/avatars/cd46fbd9b8e23b31d8c43eac25d75593.png?imageMogr2/auto-orient/strip/thumbnail/!270x270r/gravity/Center/crop/270x270/format/jpg/interlace/1/quality/80',
    'medium_avatar': 'https://img3.tapimg.com/avatars/cd46fbd9b8e23b31d8c43eac25d75593.png?imageMogr2/auto-orient/strip/thumbnail/!180x180r/gravity/Center/crop/180x180/format/jpg/interlace/1/quality/40',
    'avatar_pendant': ''}},
  'device': '小米15',
  'can_show_history': True,
  'review': {'id': 44537218,
   'score': 5,
   'played_spent': 900,
   'contents': {'text': '只能说，这游戏很多地方都很恶心人，号与号之间运气差距是巨大的，有的号做啥都很欧，打次装备就能实现转盘自由，有的号每次转盘都得氪好几个648，还有打竞技，挖宝，副本等等，欧的号真的很欧，黑的号想砸手机的心都有。另外想玩的舒服三卡足够，零氪也可以玩，这游戏收益虽然一再被削，但还是量大管饱的。最主要的是，一定要有人一起玩，一个人很难玩下去，有固定队一起配合久了是能打大氪的。附灵真的是救了这游戏，能看出来策划是努力在改变的，不管是好是坏，但真

In [25]:
for item in reviews:
    print(item)

In [34]:
gen1, gen2 = tee(reviews)
count = sum(1 for _ in gen1)

KeyboardInterrupt: 

In [32]:
count

0

In [35]:
# Manually fetch the first page to get total count
data = client.get("review/v2/list-by-app", {"app_id": 209601})
print(data["total"])

6480
